In [7]:
import os, sys
import time
import datetime
import pandas as pd
import numpy as np
import math
from math import radians, cos, sin, asin, sqrt 
import random

from scipy.spatial import ConvexHull

import folium

import geopandas as gp

import h3


In [8]:
def Convert_gps(POLYLINE):
    
    sample_trajectory=[[float(s) for s in seg.split(',')][::-1] for seg in str(POLYLINE).replace(']]','').replace('[','').split('],')]
    
    return sample_trajectory

def Convert_grid(Trajectory):
    
    sample_grid=[h3.geo_to_h3(x[0], x[1], 10) for x in Trajectory]
    
    i=1
    
    while i<len(sample_grid):
        
        if sample_grid[i]==sample_grid[i-1]:
            
            sample_grid.pop(i)
            
        else:
            
            i+=1
        
    
    return sample_grid

def Convert_Geometry(Grids):
    
    sample_geometry=list()
    
    for grid in Grids:
        
        geometry=list()
    
        for x in h3.h3_to_geo_boundary(grid):
            
            geometry.append([float(x[1]),float(x[0])])
            
        sample_geometry.append(geometry)
            
    return sample_geometry


In [9]:
df_whole=pd.read_csv('data/train.csv')

df_sample=df_whole.sample(frac=0.01)

df=df_sample.reset_index(drop=True)

df=df.iloc[:1]

df=df[['TRIP_ID','POLYLINE']]

df=df[df['POLYLINE']!='[]']

df['Trajectory']=df.apply(lambda x:Convert_gps(x['POLYLINE']),axis=1)

df['Grids']=df.apply(lambda x:Convert_grid(x['Trajectory']),axis=1)

df['Geometry']=df.apply(lambda x:Convert_Geometry(x['Grids']),axis=1)

df = df[['TRIP_ID','Grids','Geometry']]

df



,TRIP_ID,Grids,Geometry
0,1393036833620000179,"[8a39220f1d0ffff, 8a39220f1c27fff, 8a39220f1c3...","[[[-8.607852791813833, 41.15821438664012], [-8..."


In [10]:
GeoData=list()
         
for idx,row in df.iterrows():
    
    for i in range(len(row['Grids'])):
    
        GeoData.append({"type":"Feature",\
                        "id":row['Grids'][i],\
                        "geometry":{"type":"Polygon",\
                                    "coordinates":[row['Geometry'][i]]
                                   }
                       })
    
GeoJson={"type":"FeatureCollection","features":GeoData}




background = folium.Map([41.148738, -8.612091],tiles='CartoDB positron',zoom_start=12)

'''Zones'''

choropleth1 =folium.Choropleth(
    geo_data=GeoJson,
    fill_color='red',
    fill_opacity=0.5,
    name='Zone').add_to(background)

folium.LayerControl().add_to(background)

background

